# CADP tools queries 

## 2. SVL queries

In [31]:
import pandas as pd
import numpy as np 


In [22]:
file_name = "9M-5-[1-2001]-[a]-lcm.bcg"
file_path = f"../graph/bcg/{file_name}"

In [23]:
before_promo_period = "2018-09-01"
promo_period ="2018-12-01"
after_promo_period = ":2019-02-01"

In [48]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.classes_ = np.load(f'../../plots/encoders/{file_name.replace("bcg","out")}.npy')
promo_products = pd.read_csv("../../datasets/Total/promo_products.csv")
promo_products = promo_products[promo_products.ARTICLE_ID.isin(encoder.classes_)]
promo_products.ARTICLE_ID.apply(lambda x : encoder.transform([x]))
promo_products = f':{":".join(promo_products.ARTICLE_ID.astype(str).unique())}:'

### 2.1 Adoption query

In [27]:
query_name = "product_adoption.svl"
query_path = f"../queries/svl/{query_name}"
file_path = f'../graph/bcg/9M-5-[1-2001]-[a]-lcm'

In [49]:
%%bash -s $file_path $promo_products
export PATH=$PATH:$CADP/bin.x64:$CADP/com
$CADP/com/svl product_adoption.svl $1 $2 ":2018-09-01" ":2018-12-01" ":2019-02-01"


property PRODUCT_ADOPTION
 | Product adoption


[Customers] [Promotional products] Period

[124784, 521161, 1236845] [2812083228] :2018-12-01
[191426] [2812093258] :2018-12-01
[199984, 450658] [2812083228] :2018-12-01
[199984] [2812083228] :2018-12-01
[230856] [2812093258] :2018-12-01
[324441] [2812083228] :2018-12-01
[450658] [2812083228] :2018-12-01
[592510, 1160772] [2812083228] :2018-12-01
[592510] [2812083228] :2018-12-01
[596052, 1072489] [2812083808] :2018-12-01
[596052] [2812083808] :2018-12-01



###  2.2 Loyalty query 

In [88]:
%%bash -s $file_path
export PATH=$PATH:$CADP/bin.x64:$CADP/com
$CADP/com/svl product_loyalty.svl $1 ":3461025:7461024:" ":2018-12-01" ":2019-02-01"


property product_loyalty

TRUE

[Customers] [Promotional products] Period

[1044697] [3461025] :2018-12-01
[211249] [7461024] :2018-12-01
[575459, 1044697, 1163180] [3461025] :2018-12-01
[575459] [3461025] :2018-12-01



### 2.3 Pomotion only query

In [108]:
%%bash -s $file_path
export PATH=$PATH:$CADP/bin.x64:$CADP/com
$CADP/com/svl promotion_only.svl $1 ":2018-09-01" ":2018-12-01" ":2019-02-01"


property promotion_only

TRUE

[Customers] [Promotional products] Period




In [109]:
!gedit promotion_only.svl
